In [1]:
import requests
import time
import os
import re
import regex as re
import pandas as pd

from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains


from selenium import webdriver
import selenium.webdriver as webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

class Datos_linkedin:
  def __init__(self, url, puesto, empresa, lugar, logo, lapso_capturado):
    self.url = url
    self.puesto = puesto
    self.empresa = empresa
    self.lugar = lugar
    self.logo = logo
    self.lapso_capturado=lapso_capturado

In [2]:
#Accede al webdriver en pantalla completa

url = 'https://www.linkedin.com/jobs/jobs-in-mexico?trk=homepage-basic_guest_nav_menu_jobs&position=1&pageNum=0'
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
#options.add_argument('auto-open-devtools-for-tabs')
options.add_argument("--start-maximized")
driver = webdriver.Chrome(executable_path='C:\Chromedriver\chromedriver.exe', options=options)
driver.get(url)
driver.implicitly_wait(10)

C:\Users\Tole 01\AppData\Local\Temp\ipykernel_29636\659870698.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='C:\Chromedriver\chromedriver.exe', options=options)


In [5]:
input_trabajo = driver.find_elements("xpath",'//section[contains(@class,"dismissable-input typeahead-input keywords-typeahead-input text-input")]/input')
input_locacion = driver.find_elements("xpath",'//section[contains(@class,"dismissable-input typeahead-input location-typeahead-input")]/input')
boton_buscar = driver.find_elements("xpath",'//button[contains(@data-tracking-control-name,"public_jobs_jobs-search-bar_base-search-bar-search-submit")]')
input_trabajo[0].clear()
input_locacion[0].clear()
input_trabajo[0].send_keys('Full Stack')
time.sleep(1)
boton_buscar[0].send_keys('United States')

In [6]:
# Delay del objeto WebDriverWait
delay = 20

# Filtra las vacantes que tienen menos de 1 semana
boton_TiempoTranscurrido = driver.find_elements("xpath",'//button[contains(@aria-label,"Date Posted filter. Any Time filter is currently applied. Clicking this button displays all Date Posted filter options.")]')
boton_1semana = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",'//div[contains(@aria-label,"Date Posted filter options")]/div/input[contains(@id,"f_TPR-1")]')))
boton_Done_TiempoTranscurrido = driver.find_elements("xpath",'//button[contains(@data-tracking-control-name,"public_jobs_f_TPR")]')

boton_TiempoTranscurrido[0].click()
time.sleep(1)
boton_1semana.send_keys('Full Stack')
time.sleep(1)
boton_Done_TiempoTranscurrido[1].send_keys('Full Stack')
time.sleep(2)

# Filtra las vacantes que son en remoto

boton_SiteRemote = driver.find_elements("xpath",'//button[contains(@aria-label,"On-site/Remote filter. Clicking this button displays all On-site/Remote filter options.")]')
checkbox_SiteRemote = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",'//input[contains(@id,"f_WT-1")]')))
boton_Done_SiteRemote = driver.find_elements("xpath",'//button[contains(@data-tracking-control-name,"public_jobs_f_WT")]')

boton_SiteRemote[0].click()
time.sleep(1)
checkbox_SiteRemote.send_keys('Full Stack')
time.sleep(1)
boton_Done_SiteRemote[1].send_keys('Full Stack')


In [ ]:
# Obtiene el numero total de vacantes

VacantesPorScrapear = driver.find_element(By.CLASS_NAME, "results-context-header__job-count").text
num_vacantes = re.findall(r'\d+(?=[+,])?', VacantesPorScrapear)
num_vacantes=int(''.join(num_vacantes))
num_vacantes

In [ ]:
#ESTRATEGIA 1: le pica muchas veces al boton, scrollea lo mas que pueda y luego recupera la info de un jalon
num_scrolleado = 0
Objetos_linkedin=[]

while num_scrolleado <= 125:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    num_scrolleado = num_scrolleado + 1
    
    try:
        boton_scroll = driver.find_element("xpath",'//button[contains(@aria-label,"Load more results")]')
        driver.execute_script("arguments[0].click();", boton_scroll)
        print(f'La pagina se ha scrollado {num_scrolleado} veces')
        time.sleep(1)         
    except:
        pass
        print('me trabe')
        time.sleep(6)     
    

In [ ]:
# Abre la pagina de la tarjeta y se cambia a la nueva pagina para extraer info, luego regresa

card_clickable = driver.find_elements("xpath",'//a[contains(@class,"base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]")]')
card_clickable[0].click()


for i in range(0,len(card_clickable)):
    try:
        # Clickea la tarjeta y obtiene su URL
        card_clickable[i].click()
        time.sleep(2)
        URL_puesto = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",'//a[@class="topcard__link"]')))
        
        # Abre una nueva ventana que corresponde al URL de la tarjeta y cambia el driver para alla 
        main_window = driver.current_window_handle
        URL_puesto.send_keys(Keys.CONTROL + Keys.RETURN)
        pagina_original = driver.current_window_handle
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(2)
        
        # Encuentra los datos en las diferentes partes de la pagina
        URL_vacante = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",'//link[@rel="canonical"]')))
        Puesto = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",'//h1[@class="top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title"]')))
        Empresa = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",'//a[@class="topcard__org-name-link topcard__flavor--black-link"]')))
        Lugar = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",'//span[@class="topcard__flavor topcard__flavor--bullet"]')))
        URL_logo = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",'//a[@data-tracking-control-name="public_jobs_topcard_logo"]/img')))
        Timelapse_not_transformed = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",'//span[@class="posted-time-ago__text topcard__flavor--metadata" or @class="posted-time-ago__text posted-time-ago__text--new topcard__flavor--metadata"]')))
        
        try:   
            # Pasa los datos a distintos objetos, hace un append en una lista y cierra la ventana
            Objetos_linkedin.append(Datos_linkedin(URL_vacante.get_attribute('href'), Puesto.text, Empresa.text, Lugar.text, URL_logo.get_attribute('src'), Timelapse_not_transformed.text)) 
            print(f'Ya obtuve el dato {i}')
            driver.close()
            driver.switch_to.window(pagina_original)
            #time.sleep(2)
        except:
            # Si no entra en la direccion de la tarjeta, entonces imprime esto y cierra la pagina
            print(f'No pude obtener el dato {i}')
            driver.close()
            driver.switch_to.window(pagina_original)
            #time.sleep(2)
    except:
        # Si no encuentra la tarjeta, se pasa a la otra 
        print(f'No pude obtener el dato {i} por el click de la tarjeta')
        driver.switch_to.window(pagina_original)
        pass

In [ ]:
# Imprime los datos de cada objeto
print(len(Objetos_linkedin))
for j in range(len(Objetos_linkedin)):
    print(f'Objeto numero {j}')
    print(Objetos_linkedin[j].url+'\n'+Objetos_linkedin[j].puesto+'\n'+Objetos_linkedin[j].empresa+'\n'+Objetos_linkedin[j].lugar+'\n'+Objetos_linkedin[j].logo+'\n'+Objetos_linkedin[j].lapso_capturado+'\n'+'----------')

In [ ]:
# Estrategia 2: Le pica al boton y se va de 1 en 1 hasta que vuelve a picarle al boton.
URL_puesto = driver.find_elements("xpath",'//a[@class="topcard__link"]')
Puesto = driver.find_elements("xpath",'//h2[@class="top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title"]')
Empresa = driver.find_elements("xpath",'//a[@class="topcard__org-name-link topcard__flavor--black-link"]')
Lugar = driver.find_elements("xpath",'//span[@class="topcard__flavor topcard__flavor--bullet"]')
URL_logo = driver.find_elements("xpath",'//a[@data-tracking-control-name="public_jobs_topcard_logo"]/img')
Timelapse_not_transformed = driver.find_elements("xpath",'//span[@class="posted-time-ago__text topcard__flavor--metadata" or @class="posted-time-ago__text posted-time-ago__text--new topcard__flavor--metadata"]')

num_scrolleado = 0
checkpoint = [0]
Objetos_linkedin=[]

while num_scrolleado<=16:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    num_scrolleado = num_scrolleado + 1
    card_clickable = driver.find_elements("xpath",'//a[contains(@class,"base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]")]')
    
    try:
        boton_scroll = driver.find_element("xpath",'//button[contains(@aria-label,"Load more results")]')
        driver.execute_script("arguments[0].click();", boton_scroll)
        print(f'La pagina se ha scrollado {num_scrolleado} veces')
        for i in range(sum(checkpoint),(sum(checkpoint)+len(card_clickable))):
                card_clickable[i].click()
                print(i)
                Objetos_linkedin.append(Datos_linkedin(URL_puesto[0].get_attribute('href'), Puesto[0].text, Empresa[0].text, Lugar[0].text, URL_logo[0].get_attribute('src'), Timelapse_not_transformed[0].text))
                time.sleep(10)
        checkpoint.append(len(card_clickable))
        time.sleep(10)
        
    except:
        pass
        print('me trabe')
        checkpoint.append(len(card_clickable))
        time.sleep(6)        

In [ ]:
# Obtiene los datos en listas en vez de objetos

card_clickable = driver.find_elements("xpath",'//a[contains(@class,"base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]")]')

delay = 20

URL_puesto = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",'//a[@class="topcard__link"]')))
Puesto = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",'//h2[@class="top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title"]')))
Empresa = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",'//a[@class="topcard__org-name-link topcard__flavor--black-link"]')))
Lugar = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",'//span[@class="topcard__flavor topcard__flavor--bullet"]')))
URL_logo = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",'//a[@data-tracking-control-name="public_jobs_topcard_logo"]/img')))
Timelapse_not_transformed = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",'//span[@class="posted-time-ago__text topcard__flavor--metadata" or @class="posted-time-ago__text posted-time-ago__text--new topcard__flavor--metadata"]')))

lista_URL_puesto=[]
lista_Puesto=[]
lista_Empresa=[]
lista_Lugar=[]
lista_URL_logo=[]
lista_Timelapse_not_transformed=[]

for i in range(0,len(card_clickable)):
    card_clickable[i].click()
    print(i)
    try:
        time.sleep(5)
        lista_URL_puesto.append(URL_puesto.get_attribute('href'))
        lista_Puesto.append(Puesto.text)
        lista_Empresa.append(Empresa.text)
        lista_Lugar.append(Lugar.text)
        lista_URL_logo.append(URL_logo.get_attribute('src'))
        lista_Timelapse_not_transformed.append(Timelapse_not_transformed.text)        
        print(f'Ya obtuve el dato {i}')
    except:
        time.sleep(3)
        print(f'No pude obtener el dato {i}')

In [ ]:
#DIRECCIONES DE LOS DATOS (Falta modalidad, salario y descripcion)

# URL de la vacante
URL_puesto = driver.find_elements("xpath",'//a[@class="topcard__link"]')
URL_puesto[0].get_attribute('href')

# El puesto al que aplican
Puesto = driver.find_elements("xpath",'//h2[@class="top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title"]')
Puesto[0].text

# La empresa a la que pertenece el puesto
Empresa = driver.find_elements("xpath",'//a[@class="topcard__org-name-link topcard__flavor--black-link"]')
Empresa[0].text

# El tiempo que ha transcurrido desde su publicacion. Este tiene que ser tratado posteriormente para que concuerde con la fecha actual
Timelapse_not_transformed = driver.find_elements("xpath",'//span[@class="posted-time-ago__text topcard__flavor--metadata" or @class="posted-time-ago__text posted-time-ago__text--new topcard__flavor--metadata"]')
Timelapse_not_transformed[0].text

# Lugar de trabajo
Lugar = driver.find_elements("xpath",'//span[@class="topcard__flavor topcard__flavor--bullet"]')
Lugar[0].text

# URL del logo
URL_logo = driver.find_elements("xpath",'//a[@data-tracking-control-name="public_jobs_topcard_logo"]/img')
URL_logo[0].get_attribute('src')

# Son datos que contienen{ 0:Seniority, 1:Si el empleo es de tiempo completo o no, etc , 2:La funcion principal del trabajo}
Datos_extra = driver.find_elements("xpath",'//span[@class="description__job-criteria-text description__job-criteria-text--criteria"]')
Seniority = Datos_extra[0].text
Tipo_empleo = Datos_extra[1].text
Funcion_principal = Datos_extra[2].text
